# Experiment 1: Bayesian Decision Rules

## Iris Classification

In [1]:
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt

### Dataset

In [2]:
iris=datasets.load_iris()

#data对应了样本的4个特征，150行4列
print(iris.data.shape) 

#target对应样本的标签，159行1列
print(iris.target.shape)

#显示每个类别前2个样本特征
print('iris dataset: \n',iris.data[(0,1,50,51,100,101),:])
 
#显示样本特征对应的标签
print('corresponding labels: \n', iris.target[(0,1,50,51,100,101),])

(150, 4)
(150,)
iris dataset: 
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.3 3.3 6.  2.5]
 [5.8 2.7 5.1 1.9]]
corresponding labels: 
 [0 0 1 1 2 2]


In [3]:
#计算3类数据的均值和协方差矩阵
#均值
iris1 = iris.data[:20,:]
iris2 = iris.data[50:70,:]
iris3 = iris.data[100:120,:]

ave1 = np.mean(iris1,0)
ave2 = np.mean(iris2,0)
ave3 = np.mean(iris3,0)

#方差
varin1 = np.cov(iris1.T)
varin2 = np.cov(iris2.T)
varin3 = np.cov(iris3.T)

print('数据集1的均值：\n',ave1)
print('数据集2的均值：\n',ave2)
print('数据集3的均值：\n',ave3)

print('数据集1的方差：\n',varin1)
print('数据集2的方差：\n',varin2)
print('数据集3的方差：\n',varin3)

数据集1的均值：
 [5.035 3.48  1.435 0.235]
数据集2的均值：
 [5.975 2.76  4.255 1.325]
数据集3的均值：
 [6.56  2.92  5.655 2.045]
数据集1的方差：
 [[0.18239474 0.15231579 0.01976316 0.02239474]
 [0.15231579 0.16589474 0.01547368 0.02863158]
 [0.01976316 0.01547368 0.02134211 0.00502632]
 [0.02239474 0.02863158 0.00502632 0.00871053]]
数据集2的方差：
 [[0.36407895 0.14157895 0.22723684 0.06697368]
 [0.14157895 0.14252632 0.10863158 0.05105263]
 [0.22723684 0.10863158 0.20155263 0.06539474]
 [0.06697368 0.05105263 0.06539474 0.03986842]]
数据集3的方差：
 [[0.53515789 0.144      0.43336842 0.07031579]
 [0.144      0.14905263 0.11621053 0.068     ]
 [0.43336842 0.11621053 0.40997368 0.07528947]
 [0.07031579 0.068      0.07528947 0.07523684]]


In [4]:
#定义高斯函数用于计算条件概率
def Gaussfunction(data,dimension,mean,variance):
    constant = 1/(np.power(2 * np.pi,dimension/2 ) * np.sqrt(abs(np.linalg.det(variance))))
    variance_inv = np.linalg.inv(variance)
    error = data - mean
    value = constant * np.exp(-(error.T @ variance_inv @ error)/2.0)
    return value

### 最小错误决策

In [5]:
#定义两两比较的最小错误函数
#其中data为待分类的数据
#dim(ension)为数据集的维度
#m(ean) v(ariance) label = [2 1] 
def MiniErrorDecFunc(data,dimension,m1,m2,v1,v2,label,threshold):
    P_x_w1 = Gaussfunction(data,dimension,m1,v1)
    P_x_w2 = Gaussfunction(data,dimension,m2,v2)
    likelihood = P_x_w1 / P_x_w2
    decision = label[0]
    if likelihood < threshold:
        decision = label[1]
    return P_x_w1,P_x_w2,decision

### 最小风险决策

In [6]:
#公式：计算权值对阈值影响
def ValueMatrixFunc(V_matrix):
    return (V_matrix[0][1] - V_matrix[1][1]) / (V_matrix[1][0] - V_matrix[0][0])    

def MiniRiskDecFunc(data,dimension,m1,m2,v1,v2,label,threshold,VMatrix):
    P_x_w1 = Gaussfunction(data,dimension,m1,v1)
    P_x_w2 = Gaussfunction(data,dimension,m2,v2)
    likelihood = P_x_w1 / P_x_w2
    decision = label[0]
    
    threshold = ValueMatrixFunc(VMatrix) * threshold
    
    if likelihood < threshold:
        decision = label[1]
    return P_x_w1,P_x_w2,decision

### 样本分类

In [7]:
#定义计算错误率的函数
def rate_function(decision):
    error1 = abs(decision[:50] - label1)
    error2 = abs(decision[50:100] - label2)

    error1[error1>0] = 1
    error2[error2>0] = 1

    error_rate1 = sum(error1) / 50
    error_rate2 = sum(error2) / 50

    print('error_rate1 = %f error_rate2 = %f ' % (error_rate1,error_rate2))
    

In [8]:
#最小错误决策分类 阈值为1，区分1 2类  
P_x_w1 = np.zeros((100,1))
P_x_w2 = P_x_w1
decision = P_x_w1
label1 = 1
label2 = 2
threshold = 1
for i in range(100):
    [P_x_w1[i], P_x_w2[i], decision[i]] = MiniErrorDecFunc(iris.data[i,:],4,ave1,ave2,varin1,varin2,[1,2],threshold)
rate_function(decision)

error_rate1 = 0.000000 error_rate2 = 0.000000 


In [9]:
# 最小错误决策分类 阈值为1，区分1 3类
P_x_w1 = np.zeros((100,1))
P_x_w3 = P_x_w1
decision = P_x_w1
label1 = 1
label2 = 3
threshold = 1
for i in range(100):
    if i > 49 :
        [P_x_w1[i], P_x_w3[i], decision[i]] = MiniErrorDecFunc(iris.data[i+50,:],4,ave1,ave3,varin1,varin3,[1,3],threshold) 
        
    [P_x_w1[i], P_x_w3[i], decision[i]] = MiniErrorDecFunc(iris.data[i,:],4,ave1,ave3,varin1,varin3,[1,3],threshold)

rate_function(decision)

error_rate1 = 0.000000 error_rate2 = 0.000000 


In [10]:
#最小错误决策分类 阈值为1，区分1 2类
P_x_w2 = np.zeros((100,1))
P_x_w3 = P_x_w2
decision = P_x_w2
label1 = 2
label2 = 3
threshold = 1
for i in range(100):
    [P_x_w2[i], P_x_w3[i], decision[i]] = MiniErrorDecFunc(iris.data[i+50,:],4,ave2,ave3,varin2,varin3,[2,3],threshold)

rate_function(decision)

error_rate1 = 0.040000 error_rate2 = 0.000000 


In [11]:
#最小错误决策分类 阈值从1到10， 区分2 3 类
P_x_w2 = np.zeros((100,1))
P_x_w3 = P_x_w2
decision = P_x_w2
label1 = 2
label2 = 3

for threshold in range(10):
    for i in range(100):
        [P_x_w2[i], P_x_w3[i], decision[i]] = MiniErrorDecFunc(iris.data[i+50,:],4,ave2,ave3,varin2,varin3,[2,3],threshold)
    rate_function(decision)    

error_rate1 = 0.000000 error_rate2 = 1.000000 
error_rate1 = 0.040000 error_rate2 = 0.000000 
error_rate1 = 0.040000 error_rate2 = 0.000000 
error_rate1 = 0.060000 error_rate2 = 0.000000 
error_rate1 = 0.060000 error_rate2 = 0.000000 
error_rate1 = 0.060000 error_rate2 = 0.000000 
error_rate1 = 0.060000 error_rate2 = 0.000000 
error_rate1 = 0.060000 error_rate2 = 0.000000 
error_rate1 = 0.060000 error_rate2 = 0.000000 
error_rate1 = 0.060000 error_rate2 = 0.000000 


In [12]:
#最小风险决策分类 阈值为1 L12为10
#对1 2 类数据进行分类
P_x_w1 = np.zeros((100,1))
P_x_w1 = P_x_w1
decision = P_x_w1
label1 = 1
label2 = 2

#损失矩阵
Vmatrix=[[0,10],[1,0]]
threshold = 1
for i in range(100):
    [P_x_w1[i], P_x_w2[i], decision[i]] = MiniRiskDecFunc(iris.data[i,:],4,ave1,ave2,varin1,varin2,[1,2],threshold,Vmatrix)
rate_function(decision)    

error_rate1 = 0.000000 error_rate2 = 0.000000 


In [13]:
#最小风险决策分类 阈值为1 L12为10
#对1 3 类数据进行分类
P_x_w1 = np.zeros((100,1))
P_x_w3 = P_x_w1
decision = P_x_w1
label1 = 1
label2 = 3

#损失矩阵
Vmatrix=[[0,10],[1,0]]
threshold = 1
for i in range(100):
    if i > 49:
        [P_x_w1[i], P_x_w3[i], decision[i]] = MiniRiskDecFunc(iris.data[i+50,:],4,ave1,ave3,varin1,varin3,[1,3],threshold,Vmatrix)  
        
    [P_x_w1[i], P_x_w3[i], decision[i]] = MiniRiskDecFunc(iris.data[i,:],4,ave1,ave3,varin1,varin3,[1,3],threshold,Vmatrix)
    
rate_function(decision)

error_rate1 = 0.000000 error_rate2 = 0.000000 


In [14]:
#最小风险决策分类 阈值为1 L12为10
#对2 3 类数据进行分类
P_x_w2 = np.zeros((100,1))
P_x_w3 = P_x_w2
decision = P_x_w2
label1 = 2
label2 = 3

#损失矩阵L12=10
Vmatrix=[[0,10],[1,0]]
threshold = 1

for i in range(100): 
    [P_x_w2[i], P_x_w3[i], decision[i]] = MiniRiskDecFunc(iris.data[i+50,:],4,ave2,ave3,varin2,varin3,[2,3],threshold,Vmatrix)
    
rate_function(decision)

#最小风险决策分类 阈值为1 L12为100
Vmatrix=[[0,100],[1,0]]
threshold = 1

for i in range(100): 
    [P_x_w2[i], P_x_w3[i], decision[i]] = MiniRiskDecFunc(iris.data[i+50,:],4,ave2,ave3,varin2,varin3,[2,3],threshold,Vmatrix)
    
rate_function(decision)

#最小风险决策分类 阈值为10 L12为100
Vmatrix=[[0,100],[1,0]]
threshold = 10

for i in range(100): 
    [P_x_w2[i], P_x_w3[i], decision[i]] = MiniRiskDecFunc(iris.data[i+50,:],4,ave2,ave3,varin2,varin3,[2,3],threshold,Vmatrix)
    
rate_function(decision)

error_rate1 = 0.060000 error_rate2 = 0.000000 
error_rate1 = 0.160000 error_rate2 = 0.000000 
error_rate1 = 0.260000 error_rate2 = 0.000000 


# Neyman-Person决策

In [15]:
iris1 = iris.data[:50,2]
iris2 = iris.data[50:100,2]
iris3 = iris.data[100:150,2]

junzhi1 = np.mean(iris1[:20],0)
junzhi2 = np.mean(iris2[:20],0)
junzhi3 = np.mean(iris3[:20],0)

#方差
fangcha1 = np.cov(iris1[:20].T)
fangcha2 = np.cov(iris2[:20].T)
fangcha3 = np.cov(iris3[:20].T)

print('数据集1的均值：',junzhi1)
print('数据集2的均值：',junzhi2)
print('数据集3的均值：',junzhi3)

print('数据集1的方差：',fangcha1)
print('数据集2的方差：',fangcha2)
print('数据集3的方差：',fangcha3)

数据集1的均值： 1.4349999999999998
数据集2的均值： 4.255
数据集3的均值： 5.655
数据集1的方差： 0.02134210526315789
数据集2的方差： 0.20155263157894743
数据集3的方差： 0.4099736842105264


In [16]:
def new_gauss_function(data,mean,variance):
    constant = 1/(np.sqrt(2 * np.pi * variance))
    error = data - mean
    value = constant * np.exp(-(error * error)/(2.0*variance))
    return value

In [17]:
#calculate_threshold 根据最小允许的错误率和条件概率分布来计算阈值
def calculate_threshold(mini_error,m1,v1,m2,v2):
    data = m2 + mini_error * np.sqrt(v2)
    P1 = new_gauss_function(data,m1,v1)
    P2 = new_gauss_function(data,m2,v2)
    threshold = P1 / P2
    return threshold

In [18]:
#Neyman_Person决策函数定义
def Neyman_Person(data,m1,m2,v1,v2,labels,mini_error):
    threshold = calculate_threshold(mini_error,m1,v1,m2,v2)
    P_x_w1 = new_gauss_function(data,m1,v1)
    P_x_w2 = new_gauss_function(data,m2,v2)
    likelihood = P_x_w1 / P_x_w2
    decision = labels[0]
    
    if likelihood < threshold:
        decision = labels[1]
    return P_x_w1,P_x_w2,decision,threshold

In [19]:
#Neyman-Person决策 第二类错误率0.06 对1 2 类样本分类
mini_error = -1.65
threshold = 0
label1 = 1
label2 = 2
for i in range(100):
    if i < 50:
        [P_x_w1[i],P_x_w2[i],decision[i],threshold] = Neyman_Person(iris1[i],junzhi1,junzhi2,fangcha1,fangcha2,[1,2],mini_error)
    else:
        [P_x_w1[i],P_x_w2[i],decision[i],threshold] = Neyman_Person(iris2[i-50],junzhi1,junzhi2,fangcha1,fangcha2,[1,2],mini_error)    

print('threshold = ',threshold)
rate_function(decision)

threshold =  1.2348327832484011e-43
error_rate1 = 0.000000 error_rate2 = 0.100000 


In [20]:
#Neyman-Person决策 第二类错误率0.06 对1 3 类样本分类
mini_error = -1.65
label1 = 1
label2 = 3
for i in range(100):
    if i < 50:
        [P_x_w1[i],P_x_w2[i],decision[i],threshold] = Neyman_Person(iris1[i],junzhi1,junzhi3,fangcha1,fangcha3,[1,3],mini_error)
    else:
        [P_x_w1[i],P_x_w2[i],decision[i],threshold] = Neyman_Person(iris3[i-50],junzhi1,junzhi3,fangcha1,fangcha3,[1,3],mini_error)    

print('threshold = ',threshold)
rate_function(decision)

threshold =  2.5535970643281388e-101
error_rate1 = 0.000000 error_rate2 = 0.020000 


In [21]:
#Neyman-Person决策 第二类错误率0.06 对2 3类样本分类
mini_error = -1.65
label1 = 2
label2 = 3
for i in range(100):
    if i < 50:
        [P_x_w1[i],P_x_w2[i],decision[i],threshold] = Neyman_Person(iris2[i],junzhi2,junzhi3,fangcha2,fangcha3,[2,3],mini_error)
    else:
        [P_x_w1[i],P_x_w2[i],decision[i],threshold] = Neyman_Person(iris3[i-50],junzhi2,junzhi3,fangcha2,fangcha3,[1,3],mini_error)    

print('threshold = ',threshold)
rate_function(decision)

threshold =  4.151756745027747
error_rate1 = 0.280000 error_rate2 = 0.020000 
